In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# Set seed
seed = 42
torch.manual_seed(seed);

In [3]:
# Load our dataset
boston = load_boston()
train_x, test_x, train_y, test_y = train_test_split(boston.data, boston.target, random_state=seed)
scaler = MinMaxScaler()


train_x = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float)
test_x = torch.tensor(scaler.transform(test_x), dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.float).view(-1, 1)
test_y = torch.tensor(test_y, dtype=torch.float).view(-1, 1)

In [4]:
# Set some parameters
layer_size = train_x.shape[1]
lr = 0.01
epochs = 700

In [5]:
# Define model
class Regression(nn.Module):
    def __init__(self, layer_size):
        super().__init__()
        self.layer_1 = nn.Linear(layer_size, 1) # equivalent of wx + b
        
    def forward(self, x):
        return self.layer_1(x) 

In [6]:
model = Regression(layer_size) # Defines our parameters for us
loss_func = nn.MSELoss() # Define loss func
opt = optim.SGD(model.parameters(), lr=lr)

In [7]:
for epoch in range(epochs):
    # Forward pass
    model.train()
    pred = model(train_x)
    loss = loss_func(pred, train_y)
    
    # Backpropagation
    loss.backward()
    opt.step()
    opt.zero_grad()
    
    # Validate model
    model.eval()
    with torch.no_grad():
        val_pred = model(test_x)
        val_loss = loss_func(val_pred, test_y)
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} Training Loss: {loss.item()} Test Loss: {val_loss.item()}")

Epoch: 0 Training Loss: 600.7393188476562 Test Loss: 451.9837951660156
Epoch: 10 Training Loss: 221.6092987060547 Test Loss: 159.03054809570312
Epoch: 20 Training Loss: 134.75296020507812 Test Loss: 99.14453887939453
Epoch: 30 Training Loss: 110.88875579833984 Test Loss: 85.04891967773438
Epoch: 40 Training Loss: 101.0737533569336 Test Loss: 79.29972839355469
Epoch: 50 Training Loss: 94.80081176757812 Test Loss: 74.97420501708984
Epoch: 60 Training Loss: 89.76229095458984 Test Loss: 71.03230285644531
Epoch: 70 Training Loss: 85.40636444091797 Test Loss: 67.41524505615234
Epoch: 80 Training Loss: 81.56295013427734 Test Loss: 64.15592956542969
Epoch: 90 Training Loss: 78.1500015258789 Test Loss: 61.25789260864258
Epoch: 100 Training Loss: 75.11003875732422 Test Loss: 58.699703216552734
Epoch: 110 Training Loss: 72.3958969116211 Test Loss: 56.44892501831055
Epoch: 120 Training Loss: 69.96705627441406 Test Loss: 54.47059631347656
Epoch: 130 Training Loss: 67.78826904296875 Test Loss: 52.73